In [1]:
import pandas as pd
import altair as alt

In [18]:
# 1. Charger et prétraiter les données
df = pd.read_csv('dpt2020.csv', sep=';')
df = df[df['preusuel'] != '_PRENOMS_RARES']           # retirer les prénoms trop rares
df = df[df['annais'] != 'XXXX'] 
df['annais'] = df['annais'].astype(int)               # convertir l’année en entier

# 2. Définir vos typologies de prénoms

prenoms_trad = {
    'JEAN', 'PIERRE', 'MICHEL', 'CLAUDE', 'PAUL', 'ANDRÉ', 'HENRI', 'LOUIS',
    'JACQUES', 'ROBERT', 'CHRISTIAN', 'ALAIN', 'GÉRARD', 'FRANÇOIS', 'MARIE',
    'ANNE', 'CATHERINE', 'FRANÇOISE', 'MONIQUE', 'DENISE'
}

prenoms_modern = {
    'EMMA', 'LOUISE', 'JADE', 'CHLOÉ', 'INÈS', 'LÉA', 'MILA', 'LÉO', 'NOAH',
    'LIAM', 'LUCAS', 'GABRIEL', 'ARTHUR', 'HUGO', 'RAPHAËL', 'ETHAN', 'ENZO',
    'MATHIS', 'MAËL', 'NOLAN', 'LOUNA'
}
df['type'] = 'AUTRE'
df.loc[df['preusuel'].isin(prenoms_trad),    'type'] = 'TRADITIONNEL'
df.loc[df['preusuel'].isin(prenoms_modern),  'type'] = 'MODERNE'

In [20]:
# ——————————————————————————————————————————————————————————————————————————————
# Viz 1 : Carte du prénom dominant par département pour une année donnée
# ——————————————————————————————————————————————————————————————————————————————

year = 2020
dominants = (
    df[df['annais'] == year]
    .groupby(['dpt', 'preusuel'])['nombre']
    .sum()
    .reset_index()
    .sort_values('nombre', ascending=False)
    .drop_duplicates('dpt')
)

# Charger votre GeoJSON simplifié des départements (à télécharger ou fournir)
# Remplacez 'path/to/departements.geojson' par votre chemin de fichier
departements_geo = alt.Data(
    url='departements.geojson',
    format={'type': 'json', 'property': 'features'}
)

chart1 = alt.Chart(departements_geo).mark_geoshape(
    stroke='lightgray'
).transform_lookup(
    lookup='properties.code',        # champ code INSEE dans votre GeoJSON
    from_=alt.LookupData(dominants, 'dpt', ['preusuel', 'nombre'])
).encode(
    color='preusuel:N',
    tooltip=[
        alt.Tooltip('properties.nom:N',   title='Département'),
        alt.Tooltip('preusuel:N',          title='Prénom dominant'),
        alt.Tooltip('nombre:Q',            title='Nombre')
    ]
).project('mercator').properties(
    width=600, height=600,
    title=f'Prénom dominant par département en {year}'
)

In [21]:
chart1

alt.Chart(...)

In [23]:
# Filtrer les départements et types
df2 = df[
    df['dpt'].isin(['75', '85']) &
    df['type'].isin(['TRADITIONNEL', 'MODERNE'])
].copy()

# Calculer le total de naissances par année et département
totals = (
    df2
    .groupby(['annais', 'dpt'])['nombre']
    .sum()
    .reset_index()
    .rename(columns={'nombre': 'total'})
)

# Calculer le nombre par type (trad./moderne) par année et département
type_sums = (
    df2
    .groupby(['annais', 'dpt', 'type'])['nombre']
    .sum()
    .reset_index()
    .rename(columns={'nombre': 'count'})
)

# Fusionner et calculer la proportion
merged = (
    type_sums
    .merge(totals, on=['annais', 'dpt'])
)
merged['ratio'] = merged['count'] / merged['total']

# Créer le graphique normalisé
chart2 = (
    alt.Chart(merged)
    .mark_line(point=True, strokeWidth=3)
    .encode(
        x=alt.X('annais:Q',
                title='Année',
                axis=alt.Axis(labelAngle=-45, tickMinStep=5)),
        y=alt.Y('ratio:Q',
                title='Proportion de naissances',
                axis=alt.Axis(format='%')),
        color=alt.Color('type:N',
                        title='Catégorie',
                        scale=alt.Scale(range=['#1f77b4', '#ff7f0e'])),
        shape=alt.Shape('type:N', title='Catégorie'),
        tooltip=[
            alt.Tooltip('dpt:N',   title='Département'),
            alt.Tooltip('type:N',  title='Catégorie'),
            alt.Tooltip('count:Q', title='Nombre'),
            alt.Tooltip('total:Q', title='Total'),
            alt.Tooltip('ratio:Q', title='Part', format='.1%')
        ]
    )
    .properties(
        width=300, height=200,
        title='Évolution normalisée des prénoms\n(Traditionnel vs Moderne)'
    )
    .facet(
        column=alt.Column('dpt:N', title='Département')
    )
    .resolve_scale(y='independent')
)

# Pour afficher dans un notebook :
chart2


alt.FacetChart(...)

In [30]:
import pandas as pd
import altair as alt

# ————————— Préparation des données —————————
div = (
    df
    .groupby(['dpt', 'annais'])
    .agg(nb_total=('nombre', 'sum'),
         nb_prenoms=('preusuel', 'nunique'))
    .reset_index()
)
div['diversite'] = div['nb_prenoms'] / div['nb_total']

# On ne garde que les 10 départements les plus “divers” en moyenne
top10 = (
    div
    .groupby('dpt')['diversite']
    .mean()
    .nlargest(10)
    .index
    .tolist()
)
div_top10 = div[div['dpt'].isin(top10)]
div_top10['annais'] = div_top10['annais'].astype(int)

# ————————— Heatmap de base —————————
base = alt.Chart(div_top10).mark_rect().encode(
    x=alt.X('annais:O',
            title='Année',
            axis=alt.Axis(labelAngle=-45, tickMinStep=5)),
    y=alt.Y('dpt:O',
            title='Département',
            sort=top10),
    color=alt.Color('diversite:Q',
                    title='Indice de diversité',
                    scale=alt.Scale(scheme='blues')),
    tooltip=[
        alt.Tooltip('dpt:N',       title='Département'),
        alt.Tooltip('annais:O',    title='Année'),
        alt.Tooltip('diversite:Q', title='Diversité', format='.1%')
    ]
).properties(
    width=600,
    height=300,
    title='Top 10 des départements par diversité des prénoms'
)

# ————————— Slider interactif —————————
year_slider = alt.binding_range(
    min=div_top10['annais'].min(),
    max=div_top10['annais'].max(),
    step=1,
    name='Année : '
)

year_select = alt.selection_single(
    fields=['annais'],
    bind=year_slider,
    init={'annais': int(div_top10['annais'].min())},
    empty='none'
)

chart3_interact = (
    base
    .add_selection(year_select)
    .transform_filter(year_select)
    .properties(
        title='Diversité des prénoms (sélection par année)'
    )
)


# Pour afficher :
chart3         # heatmap statique top10
# chart3_interact  # heatmap interactive via slider


/var/folders/j2/kbgz9r7j3px3kmp5xj7y8sdc0000gn/T/ipykernel_18899/1049349261.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  div_top10['annais'] = div_top10['annais'].astype(int)
/var/folders/j2/kbgz9r7j3px3kmp5xj7y8sdc0000gn/T/ipykernel_18899/1049349261.py:56: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  year_select = alt.selection_single(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'